Fork from [Two Sigma Financial News Competition Official Getting Started Kernel](https://www.kaggle.com/dster/two-sigma-news-official-getting-started-kernel)

Translate "Two Sigma Financial News Competition Official Getting Started Kernel" to Japanese

# Two Sigma Financial News Competition の始め方
## 導入
このコンペティションでは、市場の状態やニュース記事に基づいて株式がどのように変化するかを予測する。
学習用のデータには、長期間の取引日1日ごとの更新された市場状況と、最後の取引日以降に発行された一連のニュース記事と、影響を受けた株式や感情分析がある。
この情報を使用して、各株式が将来10営業日に増減するかどうかを予測する。 
これらの予測を行うと、次の取引日に進むことができる。

通常のKaggleコンペティションと異なる点:
- SubmissionはKaggleカーネルからのみ可能、他のデータソース、GPU、またはインターネットアクセスは使用できない。
- **2ステージコンペティション**である。ステージ1でカーネルを編集してモデルを改良する。ステージ2の開始時にカーネルがロックされ、そのカーネル使用して6ヶ月感の現実のデータに対して予測とスコアリングを行う。
- データの取扱には、**`kaggle.competitions.twosigmanews`**というPythonモジュールを使用しなければならない。これは、現在の取引日の予測を行うために将来のデータを使用していないことを保証するために情報の流れを制御する目的で導入されている。

## このスターターカーネルでは、** `twosigmanews` **モジュールを使用してトレーニングデータを取得し、テスト機能を取得し、予測を行い、提出ファイルを書き込む方法を示す。
## 忙しい人向けのコード例
```python
# 専用モジュールのインポート
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()

# トレーニングデータの取得
# market_train_df が市場データ
# news_train_df がニュースデータ
(market_train_df, news_train_df) = env.get_training_data()

# モデルの学習
# 各自、自由にモデルを構築しよう
train_my_model(market_train_df, news_train_df)

# モデルによる予測、スコア計算
# 上で構築したモデルを用いて、将来のデータを予測しよう
for (market_obs_df, news_obs_df, predictions_template_df) in env.get_prediction_days():
     predictions_df = make_my_predictions(market_obs_df, news_obs_df, predictions_template_df)
     env.predict(predictions_df)
  
# env.predictの結果を出力
env.write_submission_file()
```
`train_my_model`、 `make_my_predictions`は各々で作成する

## 詳細な導入
まずは、モジュールをインポートして環境構築しよう。

In [ ]:
from kaggle.competitions import twosigmanews
# make_env()はカーネル中で1度しか呼び出せないよ、データをなくさないように注意してね！（変数に入れてとっておいてね！）
env = twosigmanews.make_env()
print('できた！')

## **`get_training_data`** 関数

Return: 次のトレーニング用データフレームのタプルを返す。
- `market_train_df`:トレーニング用、市場データのDataFrame
- `news_train_df`:トレーニング用、ニュースデータのDataFrame

このデータフレームには、2007年2月から2016年12月までのすべての市場データとニュースデータが含まれている。
詳細は、[コンペティションのDataタブ](https://www.kaggle.com/c/two-sigma-financial-news/data)を参照すること。
列名は、各DataFrameに含まれている。

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

In [ ]:
market_train_df.head()

In [ ]:
market_train_df.tail()

In [ ]:
news_train_df.head()

In [ ]:
news_train_df.tail()

## `get_prediction_days` 関数

「予測日」（取引日）ごとにループし、最後に受け取ったデータ以降に発生したすべての市場およびニュースの観測結果を提供するジェネレータである。
将来を予測をするための** `predict` **関数を呼び出すと、次の予測日に進むことができる。

生成されるデータ：
- 直前の`get_prediction_days`でデータが生成された後に、`predict`関数が正常に呼び出されていれば、以下のタプルを生成する。
    - `market_observations_df`: 次の予測日の市場の観測データのDataFrame
    - `news_observations_df`: 次の予測日のニュースの観測データのDataFrame
    - `predictions_template_df`: `confidenceValue = 0`で初期化された`assetCode`と`confidenceValue`の列名を持つDataFrame、`predict`関数に渡すためのフォーマットになっている。
- 直前のget_prediction_daysでデータが生成された後に、`predict`関数が呼び出されていないときは`None`を生成する。

In [ ]:
# get_prediction_days（）で生成されたデータを1回だけイテレーションできる
# イテレーションが始まったら、データをなくさないように注意してね！（中のデータを変数に入れてとっておいてね！）
days = env.get_prediction_days()

In [ ]:
(market_obs_df, news_obs_df, predictions_template_df) = next(days)

In [ ]:
market_obs_df.head()

In [ ]:
news_obs_df.head()

In [ ]:
predictions_template_df.head()

`get_prediction_days`関数で取得した日の予測をせずに、次の予測日に進むとエラーが発生するので注意が必要。

In [ ]:
next(days)

### **`predict`**関数
現在の予測日の予測結果を格納する。
引数には、`get_prediction_days`から返された` predictions_template_df`と同じフォーマットのDataFrameを入れる。

引数:
`predictions_df` ：次の列を持つDataFrame
- `assetCode` ： 市場資産のコード。
- `confidenceValue` ：資産が10取引日以内に増減するかどうかの予測結果。この値は `[-1.0, 1.0]` の範囲内でなければならない。

入力引数の`predictions_df`は、`get_prediction_days`から返された`predictions_template_df`で与えられた行をすべて含んでいなければならない。
`predict`関数では検証しないが、`assetCode`が足りなかったり、余分な`assetCode`がある場合は、投稿が失敗する。

とりあえず最初の予測日にランダムな予測をしてみる：

In [ ]:
import numpy as np
def make_random_predictions(predictions_df):
    # [-1, 1]の間のランダムな値をpredictions_dfに入れる
    predictions_df.confidenceValue = 2.0 * np.random.rand(len(predictions_df)) - 1.0

In [ ]:
make_random_predictions(predictions_template_df)
env.predict(predictions_template_df)

続いて、次の予測日にもう一度ランダムな予測をしてみる：

In [ ]:
(market_obs_df, news_obs_df, predictions_template_df) = next(days)

In [ ]:
market_obs_df.head()

In [ ]:
news_obs_df.head()

In [ ]:
predictions_template_df.head()

In [ ]:
make_random_predictions(predictions_template_df)
env.predict(predictions_template_df)

## ループで全予測日に対して予測する
`For`文ですべての予測日をループして、ランダムな予測をしてみる。
`days`ジェネレータ（`get_prediction_days`から返される）は、最終日に達するとイテレーションが止まる。

In [ ]:
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    make_random_predictions(predictions_template_df)
    env.predict(predictions_template_df)
print('できた！')

## **`write_submission_file`** 関数

`predict`関数で予測した結果を現在のワーキングディレクトリにCSV形式（`submission.csv`）で出力する。

In [ ]:
env.write_submission_file()

In [ ]:
# 提出用の結果ファイルができた！
import os
print([filename for filename in os.listdir('.') if '.csv' in filename])

ヘルプメッセージにも書かれているが、`write_submission_file`関数は呼び出してもそれ自体で**コンペティションに結果が提出されるわけではない**。
これの関数は、`submission.csv`ファイルをカーネルの出力の一部として書き出すために使用される。
コンペティションに提出するためには、以下の手順を行わなければならない。
1. カーネルを**Commit**（右上のボタン）する。
2. カーネルのエディタの外側にあるKernel Version'sのOutputタブの中から出力した`submission.csv`を探す。
3. "Submit to Competition"をクリックする
ステージ2でのカーネルを再実行では、提出された結果にリンクされたKernel Version（"Commit"を押すと生成される）が使用される。

## コードを再実行するときはカーネルを再起動する
不正行為を防止のために、カーネルの実行ごとに1度だけ`make_env`を呼び出したり、`get_prediction_days`によるイテレーションをしたりすることができる。
しかし、普通はモデル構築、予測のときにちょっとした変更による試行錯誤を何度もすると思われる。
残念ながら、単にコードを再実行しようとしたり、ブラウザのページを再読込したりしても、前に実行していたときと同じカーネル実行セッションで実行されていて、`twosigmanews`モジュールにエラーを返される。
この問題を回避するには、カーネルエディタの一番下にあるConsoleタブの再起動ボタンを押すことで、実行セッションを再起動する必要がある。
![Restart button](https://i.imgur.com/hudu8jF.png)